# EEMD + LSTM

## Data Description

Input file: Historical Product Demand.csv

Description: CSV data file containing product demand for encoded product id's

Size of Data: (1048575, 5)

Features: Product_Code, Warehouse, Product_Category, Date, Order_Demand

Period: 2012-01-01 ~ 2017-01-09


---

In [2]:
# DataFrame
import pandas as pd
import numpy as np
import random
from datetime import datetime, date

# Preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

# Save the log
import os

# EEMD
from PyEMD import EEMD

# LSTM
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Metric 
# Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

## Data Explore

In [3]:
# Data Loading
df = pd.read_csv('HPD_0416.csv')
# convert the string to the datetype
#df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df

,Date,Product_Code,Product_Category,Order_Demand
0,2012-01-05,Product_0025,Category_005,1600.0
1,2012-01-06,Product_0025,Category_005,1000.0
2,2012-01-07,Product_0025,Category_005,0.0
3,2012-01-08,Product_0025,Category_005,0.0
4,2012-01-09,Product_0025,Category_005,0.0
...,...,...,...,...
14551,2016-12-23,Product_2004,Category_005,0.0
14552,2016-12-24,Product_2004,Category_005,0.0
14553,2016-12-25,Product_2004,Category_005,0.0
14554,2016-12-26,Product_2004,Category_005,2000.0


In [4]:
print(df.info())
print('-------------------------')
print("")
print("The Number of unique")
print('-------------------------')
print('Product code:\t', df.Product_Code.nunique())
print('Category:\t', df.Product_Category.nunique())
print('-------------------------')
print("The Product Code:")
print("")
for i, code in enumerate(df['Product_Code'].unique()):
    print(i+1, code)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14556 entries, 0 to 14555
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              14556 non-null  datetime64[ns]
 1   Product_Code      14556 non-null  object        
 2   Product_Category  14556 non-null  object        
 3   Order_Demand      14556 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 455.0+ KB
None
-------------------------

The Number of unique
-------------------------
Product code:	 8
Category:	 5
-------------------------
The Product Code:

1 Product_0025
2 Product_0739
3 Product_0901
4 Product_1154
5 Product_1248
6 Product_1295
7 Product_1378
8 Product_2004


---

## EEMD
    * 시계열 그래프를 ensembled IMF (앙상블 내재모드 함수)로 분해
    * n 개의 eIMFs와  1개의 Residual 생성

In [5]:
# 수요 그래프를 n개의 앙상블된 내재모드함수(IMF)로 분해
# 그래프의 변동성이 클수록, IMF의 개수 증가
def eemd_fit(df, trials=100, max_imf=-1):
    
    # Define signal
    t = np.array(df['Date']) # 날짜
    s = np.array(df['Order_Demand']) # 수요량
    
    # EEMD 객체 생성
    eemd = EEMD(trials=trials) # trials: EMD 횟수
    
    # 극값을 감지하는 방법으로 parabolic 방법을 선택
    emd = eemd.EMD
    emd.extrema_detection="parabol"
    
    # eIMFs로 분해
    eIMFs = eemd.eemd(s, t, max_imf=max_imf) # max_imf: IMF 제한 개수(-1: 없음)
    nIMFs = eIMFs.shape[0] # eIMF의 개수
    
    # 분해된 eIMFs와 잔차를 변수에 할당
    imfs, residue = eemd.get_imfs_and_residue()
    
    # 앙상블 IMFs 들의 DataFrame 생성
    all_eIMFs_df = pd.DataFrame(eIMFs).transpose()
    all_eIMFs_df[nIMFs] = residue # residue 열 마지막 열로 추가
    #all_eIMFs_df.set_index(df['Date'], inplace=True) # 날짜를 index로 setting
    all_eIMFs_df.insert(0, 'Date', df['Date']) # Date 열 추가
    
    return all_eIMFs_df, nIMFs # eIMF+Residue들로 이루어진 df, eIMF(Residue포함)의 개수

### eIMFs 데이터프레임 추출

In [6]:
# eIMF들을 추출하여, Date와 y로 이루어진 데이터프레임 추출하고 딕셔너리에 저장
def extract_eIMFs(all_eIMFs_df, nIMFs):
    all_eIMFs_dict = {}
    # IMF개수+Residue(1) 만큼 반복
    for i in range(nIMFs+1):
        tmp_df = all_eIMFs_df[['Date', i]] # n번째 eIMF에 해당하는 날짜와 값 추출
        tmp_df.columns=['Date', 'y'] # i -> y 로 열이름 변경
        all_eIMFs_dict[f'eIMFs_{i}'] = tmp_df # n번째 eIMF 정보(마지막은 Residue) 딕셔너리에 저장
        
    return all_eIMFs_dict # {eIMFs_1: df1, eIMFs_2: df2, ...}

### Split the train and test set
- Input
     data: dataframe with dates and Demand data
     
- output
    - train:  2012-01-01 ~ 2015-06/30 
    - Valid:  2015-07-01 ~ 2015-12-31
    - test :  2016-01-01 ~ 2017-01-06 
    
     
- time_steps: # of the input time steps 
- for_periods: # of the output time steps 

In [7]:
def ts_train_val_test(eIMF_df, time_steps): 

    ts_train_end = len(eIMF_df[eIMF_df['Date']<'2015-07-01']) # train 데이터 종료 인덱스
    ts_val_end = len(eIMF_df[eIMF_df['Date']<'2016-01-01']) # validation 데이터 종료 인덱스
    ts = eIMF_df.filter(['y']).values # y(수요량) 값
    
    # Minmax로 0~1 사이에 값이 오도록 정규화
    sc = MinMaxScaler() # 객체 생성
    ts_scaled = sc.fit_transform(ts) # 전체 y값 정규화
    
    # Train Data
    ts_train_scaled = ts_scaled[:ts_train_end,:]

    X_train = [] 
    y_train = []
    for i in range(time_steps, ts_train_end): 
        X_train.append(ts_train_scaled[i-time_steps:i,0]) # time steps 만큼 sliding window
        y_train.append(ts_train_scaled[i,0])

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # Reshape X_train for LSTM -> (batch_size, time_steps, features)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

    # Validation Data
    ts_val_scaled = ts_scaled[ts_train_end : ts_val_end, :]

    X_val = []
    y_val = []
    for i in range(time_steps, len(ts_val_scaled)):
        X_val.append(ts_val_scaled[i-time_steps : i, 0])
        y_val.append(ts_val_scaled[i, 0])

    X_val = np.array(X_val)
    y_val = np.array(y_val)

    # Reshape X_val for LSTM -> (batch_size, time_steps, features)
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1],1))
    
    # Test Data
    ts_test_scaled = ts_scaled[ts_val_end:,:]

    X_test = []
    y_test = eIMF_df.iloc[ts_val_end+time_steps:,:]
    y_test.loc[:, 'y_norm'] = ts_test_scaled[time_steps:].reshape(-1).copy()

    for i in range(time_steps, len(ts_test_scaled)):
        X_test.append(ts_test_scaled[i-time_steps : i, 0])
    
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
    
    return X_train, y_train, X_val, y_val, X_test, y_test, sc

### LSTM

In [8]:
def LSTM_model(X_train, y_train, X_val, y_val, X_test, sc, epochs=10):
    # LSTM 모델 객체 생성
    my_LSTM_model = Sequential() 
    
    # 첫 번째 LSTM 레이어 구성
    # 활성화 함수는 ReLU를 사용하며, return_sequences=True로 지정하여 다음 LSTM 레이어의 입력으로 사용할 수 있도록 함
    my_LSTM_model.add(LSTM(512, activation='relu',return_sequences=True, input_shape=(X_train.shape[1],1)))
    
    # 두 번째 LSTM 레이어 구성
    # 활성화 함수는 ReLU를 사용하며, return_sequences=False로 지정하여 마지막 LSTM 레이어임을 나타냄
    my_LSTM_model.add(LSTM(256, activation = 'relu',return_sequences=False))
    
    # Fully connected 레이어들 추가
    # 마지막 레이어에서는 출력의 unit 개수를 1로 설정하여 1개의 값을 출력
    my_LSTM_model.add(Dense(128))
    my_LSTM_model.add(Dense(64))
    my_LSTM_model.add(Dense(32))
    my_LSTM_model.add(Dense(1))
    
    # 모델 컴파일
    my_LSTM_model.compile(optimizer = "Adam", # Adam optimizer 사용
                         loss = 'mean_squared_error', # 손실 함수로는 평균 제곱 오차 사용
                          metrics=['mape','mae']) # 성능 지표로는 MAPE와 MAE를 사용
    #조기종료 조건
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # 모델 Fitting
    my_LSTM_model.fit(X_train, # 입력 데이터
                      y_train, # 출력 데이터
                      epochs = epochs, # epoch 수
                      batch_size = 16, # batch size
                      validation_data=(X_val, y_val),
                      callbacks=[early_stopping],# validation에 따른 조기종료
                      verbose = 1) # 학습 상태를 출력
    
    # Test 데이터 예측
    LSTM_prediction = my_LSTM_model.predict(X_test) # 예측값 얻기
    LSTM_prediction_normalized = LSTM_prediction # 예측값을 저장하되, normalize된 값 저장
    LSTM_prediction = sc.inverse_transform(LSTM_prediction) # denormalize된 예측값 저장
    
    # 모델 객체와 예측값 반환
    return my_LSTM_model, LSTM_prediction, LSTM_prediction_normalized

### EEMD+LSTM

In [9]:
def EEMD_LSTM(all_eIMFs_dict, time_steps, epochs):

    model_dict = {}
    pred_dict = {}
    
    # 모든 eIMF에 대해 LSTM 모델 학습 및 예측 실행
    for i in all_eIMFs_dict.keys():
        print(f'--------Total: 0~{len(all_eIMFs_dict)-1} eIMFs, Now: {i} --------')
        
        # 현재 eIMF 데이터 가져오기
        eIMF_df = all_eIMFs_dict[i]
        
        # 학습 데이터와 테스트 데이터 분리
        X_train, y_train, X_val, y_val, X_test, y_test, sc = ts_train_val_test(eIMF_df, time_steps)
        
        # LSTM 모델 학습 및 저장
        my_LSTM_model, LSTM_prediction, LSTM_prediction_normalized = LSTM_model(X_train, y_train, X_val, y_val, X_test, sc, epochs)
        model_dict[i] = my_LSTM_model # 딕셔너리에 모델 정보 저장
        
        # 예측 결과 저장
        y_test.reset_index(drop=True, inplace=True)
        pred_df = pd.DataFrame({'Pred': LSTM_prediction.reshape(-1) ,'Pred_norm': LSTM_prediction_normalized.reshape(-1)})
        res_df = pd.concat([y_test, pred_df], axis=1)
        res_df.set_index('Date', inplace=True)
        res_df = res_df.resample('D').first() # 증강된 데이터가 아닌, Actual값들과 비교
        pred_dict[i] = res_df
        
    # 모델과 예측값 딕셔너리 반환
    return model_dict, pred_dict

## Plot the result

In [10]:
def actual_pred_plot(product_code, pred_dict, all_result_df, metric_df, normalize=False):
    today = date.today()
    """
    Plot the actual vs predition and save the figure in the given directory
    """
    pred_dict['all_result'] = all_result_df
    
    save_path = os.path.join("Result", "EEMD+LSTM_Result", product_code+f'_{today.month:02d}{today.day:02d}')
    if normalize: save_path += "_normalized"
        
    for i, pred_df in enumerate(pred_dict.values()):
        img_n = len(pred_dict)
        title = f"Pred Actual Plot - ({i+1}/{len(pred_dict)-1})'s eIMF"
        actual = pred_df['y']
        pred = pred_df['Pred']
        save_name = f'{product_code}_eIMF_{i+1}'
        if i == img_n-1: # All result
            title = f"{product_code}-All Result"
            save_name = f'{product_code}_all_result'
        if normalize:
            title += "(Normalized)"
            actual = pred_df['y_norm']
            pred = pred_df['Pred_norm']
            
        plt.figure(figsize=(16, 8))
        plt.title(title, fontsize=20)
        plt.xlabel("Time", fontsize=14)
        plt.ylabel("Order Demand", fontsize=14)
        plt.plot(actual, label ='Actual', alpha=0.6)
        plt.plot(pred, label='Prediction', alpha=0.8)
        plt.legend(loc="upper right")
        
        # Plot 결과 저장
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        # save the figure
        today_date = f'_{today.month:02d}{today.day:02d}'
        plt.savefig(os.path.join(save_path, save_name+'.png'))
    metric_df.to_csv(os.path.join(save_path, f'{product_code}_Metric.csv'))
    del pred_dict['all_result']
        
    plt.close('all') # close all figures to free up memory

## Metrics

In [11]:
# Model Metric
def mase(training_series, testing_series, prediction_series):
    n = training_series.shape[0]
    d = np.abs(np.diff(training_series)).sum() / (n-1)
    
    errors = np.abs(testing_series - prediction_series)
    return errors.mean() / d

# Model Metric
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

def nrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, squared=False)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

In [12]:
def calculate_metrics(pred_df, normalize):
    # 계산된 메트릭을 저장하기 위해 데이터프레임 초기화
    metric_df = pd.DataFrame(columns=['MAPE', 'RMSE', 'MAE', 'NRMSE'])

    # 정규화 옵션이 True인 경우 정규화된 데이터 사용, 그렇지 않으면 원래 데이터 사용
    if normalize:
        actual = pred_df['y_norm']
        pred = pred_df['Pred_norm']
    else:
        actual = pred_df['y']
        pred = pred_df['Pred']

    # 메트릭 계산
    # MASE = mase(np.array(train_series), np.array(actual), pred) 
    MAPE = mape(actual, pred) 
    RMSE = mean_squared_error(actual, pred)**0.5 
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    # RMSLE = mean_squared_log_error(actual, pred)**0.5 

    # 계산된 메트릭을 데이터프레임에 추가
    tmp_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)]})

    # 메트릭 데이터프레임에 결과 추가
    metric_df = pd.concat([metric_df, tmp_df])
    return metric_df

---

## Check the Result

In [13]:
def make_metric_df(product_code, pred_dict, all_result_df, normalize):
    today = date.today()

    metric_df = pd.DataFrame(columns=['MAPE', 'RMSE', 'MAE', 'NRMSE'])
    for i, pred_df in pred_dict.items():
        imf_df = calculate_metrics(pred_df, normalize=normalize)
        metric_df = pd.concat([metric_df, imf_df])
    
    imf_idx = pd.Index(['eIMF_'+str(i+1) for i in range(len(pred_dict))]) # changed result_dict to pred_dict
    metric_df.index = imf_idx # Assign the created index to metric_df
    metric_df = pd.concat([metric_df, calculate_metrics(all_result_df, normalize=normalize)], axis=0)
    metric_df = metric_df.rename(index={metric_df.index[-1]: 'All'}) # 마지막 행은 all
    
    return metric_df

In [14]:
def make_all_result_df(pred_dict):
    all_df = pd.DataFrame()
    for tmp_df in pred_dict.values():
        all_df = pd.concat([all_df, tmp_df], axis=1)
    pred_df = all_df['Pred'].sum(axis=1)
    actual_df = all_df['y'].sum(axis=1)
    
    all_result_df = pd.DataFrame({'Pred': pred_df, 'y': actual_df})
    all_result_df.loc[all_result_df['Pred']<0, 'Pred']=0 # 음수 예측 값은 0으로 대치
    
    # 날짜(Date) 열은 정규화하지 않으므로 제외
    result_norm = all_result_df[['Pred', 'y']]
    
    # MinMaxScaler를 이용하여 정규화합니다.
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(result_norm)
    
    # 정규화된 데이터를 데이터 프레임에 반영합니다.
    all_result_df['Pred_norm'] = normalized_data[:,0]
    all_result_df['y_norm'] = normalized_data[:,1]
    return all_result_df

In [15]:
def execute_EEMD_LSTM(product_code, eemd_trials=100, time_steps=30, epochs=20):

    product_code = product_code # 예측하고자 하는 코드 입력
    product_df = df[df['Product_Code']== product_code].reset_index(drop=True)
    
    # EEMD 수행
    all_eIMFs_df, nIMFs = eemd_fit(product_df, eemd_trials)
    # EEMD 결과에서 각 eIMFs' DF 추출
    all_eIMFs_dict = extract_eIMFs(all_eIMFs_df, nIMFs)
    # EEMD+LSTM 실행
    model_dict, pred_dict = EEMD_LSTM(all_eIMFs_dict, time_steps, epochs) #dictionary, time_steps, epochs
    all_result_df = make_all_result_df(pred_dict)
    
    metric_df_norm = make_metric_df(product_code, pred_dict, all_result_df, True)
    metric_df = make_metric_df(product_code, pred_dict, all_result_df, False)
    
    actual_pred_plot(product_code, pred_dict, all_result_df, metric_df_norm, True)
    actual_pred_plot(product_code, pred_dict, all_result_df, metric_df, False)
    
    return metric_df

---

## Whole Process
    - product_code에 str으로 예측하고자 하는 코드를 입력
    - ['Product_0025', 'Product_0739', 'Product_0901', 'Product_1154',
       'Product_1248', 'Product_1295', 'Product_1378', 'Product_2004']

In [16]:
import time
start_time = time.time()

execute_EEMD_LSTM('Product_0025')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~11 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 24s 277ms/step - loss: 0.0102 - mape: 14.4429 - mae: 0.0637 - val_loss: 0.0102 - val_mape: 3228362.7500 - val_mae: 0.0664
Epoch 2/20
78/78 [==============================] - 19s 246ms/step - loss: 0.0049 - mape: 11.5508 - mae: 0.0501 - val_loss: 0.0095 - val_mape: 2883382.0000 - val_mae: 0.0582
Epoch 3/20
78/78 [==============================] - 19s 240ms/step - loss: 0.0046 - mape: 11.0352 - mae: 0.0480 - val_loss: 0.0099 - val_mape: 3189614.7500 - val_mae: 0.0638
Epoch 4/20
78/78 [==============================] - 19s 243ms/step - loss: 0.0048 - mape: 11.3888 - mae: 0.0494 - val_loss: 0.0090 - val_mape: 3001040.0000 - val_mae: 0.0545
Epoch 5/20
78/78 [==============================] - 20s 255ms/step - loss: 0.0048 - mape: 11.2835 - mae: 0.0491 - val_loss: 0.0093 - val_mape: 2897717.5000 - val_mae: 0.0567
Epoch 6/20
78/78 [==============================] - 20s 250ms/step - loss: 0.0047

78/78 [==============================] - 19s 244ms/step - loss: 8.3236e-05 - mape: 1.7081 - mae: 0.0070 - val_loss: 3.8005e-04 - val_mape: 622714.5625 - val_mae: 0.0159
Epoch 20/20
11/11 [==============================] - 2s 118ms/step
--------Total: 0~11 eIMFs, Now: eIMFs_3 --------
Epoch 1/20
78/78 [==============================] - 23s 263ms/step - loss: 0.0209 - mape: 27.6100 - mae: 0.1068 - val_loss: 0.0122 - val_mape: 1770523.8750 - val_mae: 0.0848
Epoch 2/20
78/78 [==============================] - 19s 245ms/step - loss: 0.0032 - mape: 11.2564 - mae: 0.0451 - val_loss: 0.0036 - val_mape: 779590.1250 - val_mae: 0.0402
Epoch 3/20
78/78 [==============================] - 20s 253ms/step - loss: 0.0012 - mape: 6.9159 - mae: 0.0278 - val_loss: 0.0016 - val_mape: 581365.4375 - val_mae: 0.0299
Epoch 4/20
78/78 [==============================] - 23s 297ms/step - loss: 4.2732e-04 - mape: 4.0420 - mae: 0.0161 - val_loss: 0.0011 - val_mape: 553687.2500 - val_mae: 0.0247
Epoch 5/20
78/78 [==

78/78 [==============================] - 19s 240ms/step - loss: 1.8838e-05 - mape: 8942.6816 - mae: 0.0034 - val_loss: 4.3481e-05 - val_mape: 0.9785 - val_mae: 0.0057
Epoch 8/20
11/11 [==============================] - 1s 115ms/step
--------Total: 0~11 eIMFs, Now: eIMFs_7 --------
Epoch 1/20
78/78 [==============================] - 21s 242ms/step - loss: 0.0129 - mape: 12.7843 - mae: 0.0638 - val_loss: 0.0014 - val_mape: 11.9817 - val_mae: 0.0326
Epoch 2/20
78/78 [==============================] - 19s 240ms/step - loss: 3.6636e-04 - mape: 3.2033 - mae: 0.0160 - val_loss: 0.0023 - val_mape: 13.1881 - val_mae: 0.0478
Epoch 3/20
78/78 [==============================] - 19s 241ms/step - loss: 2.7543e-04 - mape: 2.7719 - mae: 0.0139 - val_loss: 0.0020 - val_mape: 12.6915 - val_mae: 0.0448
Epoch 4/20
11/11 [==============================] - 1s 115ms/step
--------Total: 0~11 eIMFs, Now: eIMFs_8 --------
Epoch 1/20
78/78 [==============================] - 22s 247ms/step - loss: 0.0264 - mape: 

78/78 [==============================] - 19s 240ms/step - loss: 1.0634e-06 - mape: 0.1697 - mae: 7.7473e-04 - val_loss: 2.2898e-04 - val_mape: 8.0186 - val_mae: 0.0139
Epoch 14/20
11/11 [==============================] - 1s 113ms/step
--------Total: 0~11 eIMFs, Now: eIMFs_11 --------
Epoch 1/20
78/78 [==============================] - 21s 242ms/step - loss: 0.0253 - mape: 421544.1562 - mae: 0.1216 - val_loss: 0.0187 - val_mape: 28.1791 - val_mae: 0.1125
Epoch 2/20
78/78 [==============================] - 19s 238ms/step - loss: 0.0185 - mape: 360864.9375 - mae: 0.1082 - val_loss: 0.0186 - val_mape: 27.8421 - val_mae: 0.1122
Epoch 3/20
78/78 [==============================] - 19s 240ms/step - loss: 0.0184 - mape: 370909.0312 - mae: 0.1082 - val_loss: 0.0193 - val_mape: 26.4861 - val_mae: 0.1132
Epoch 4/20
78/78 [==============================] - 19s 237ms/step - loss: 0.0183 - mape: 361392.6875 - mae: 0.1074 - val_loss: 0.0187 - val_mape: 28.5563 - val_mae: 0.1128
Epoch 5/20
11/11 [=====

In [17]:
start_time = time.time()

execute_EEMD_LSTM('Product_0739')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~10 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 21s 244ms/step - loss: 0.0141 - mape: 423668.2500 - mae: 0.0764 - val_loss: 0.0143 - val_mape: 23.3202 - val_mae: 0.0822
Epoch 2/20
78/78 [==============================] - 19s 241ms/step - loss: 0.0074 - mape: 386038.8438 - mae: 0.0562 - val_loss: 0.0150 - val_mape: 24.9824 - val_mae: 0.0869
Epoch 3/20
78/78 [==============================] - 19s 240ms/step - loss: 0.0075 - mape: 395753.0000 - mae: 0.0567 - val_loss: 0.0144 - val_mape: 23.6222 - val_mae: 0.0829
Epoch 4/20
11/11 [==============================] - 2s 117ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_1 --------
Epoch 1/20
78/78 [==============================] - 21s 243ms/step - loss: 0.0147 - mape: 18.2347 - mae: 0.0797 - val_loss: 0.0183 - val_mape: 3236830.0000 - val_mae: 0.1007
Epoch 2/20
78/78 [==============================] - 19s 241ms/step - loss: 0.0066 - mape: 13.3804 - mae: 0.0558 - val_loss: 0.0180 - val_mape: 

Epoch 4/20
78/78 [==============================] - 22s 280ms/step - loss: 4.9624e-04 - mape: 26966.7598 - mae: 0.0170 - val_loss: 6.1938e-04 - val_mape: 7.3892 - val_mae: 0.0211
Epoch 5/20
78/78 [==============================] - 22s 281ms/step - loss: 2.0457e-04 - mape: 10823.3691 - mae: 0.0106 - val_loss: 5.7515e-04 - val_mape: 7.1507 - val_mae: 0.0204
Epoch 6/20
78/78 [==============================] - 22s 281ms/step - loss: 1.9917e-04 - mape: 6416.8330 - mae: 0.0110 - val_loss: 2.3754e-04 - val_mape: 3.6607 - val_mae: 0.0126
Epoch 7/20
78/78 [==============================] - 22s 279ms/step - loss: 1.6018e-04 - mape: 7061.6582 - mae: 0.0098 - val_loss: 4.6341e-04 - val_mape: 4.5859 - val_mae: 0.0174
Epoch 8/20
78/78 [==============================] - 22s 278ms/step - loss: 1.4320e-04 - mape: 6691.0093 - mae: 0.0092 - val_loss: 3.2555e-04 - val_mape: 5.1199 - val_mae: 0.0150
Epoch 9/20
78/78 [==============================] - 22s 279ms/step - loss: 1.4269e-04 - mape: 6058.7515 - ma

78/78 [==============================] - 23s 294ms/step - loss: 3.3696e-04 - mape: 3.6725 - mae: 0.0157 - val_loss: 2.4850e-04 - val_mape: 1.5076 - val_mae: 0.0137
Epoch 3/20
78/78 [==============================] - 21s 273ms/step - loss: 3.6498e-04 - mape: 3.6503 - mae: 0.0156 - val_loss: 1.7943e-04 - val_mape: 1.2438 - val_mae: 0.0116
Epoch 4/20
78/78 [==============================] - 21s 274ms/step - loss: 2.9119e-04 - mape: 3.2047 - mae: 0.0138 - val_loss: 8.1785e-04 - val_mape: 2.7246 - val_mae: 0.0261
Epoch 5/20
78/78 [==============================] - 20s 259ms/step - loss: 1.5735e-04 - mape: 2.4263 - mae: 0.0104 - val_loss: 8.0767e-05 - val_mape: 0.8563 - val_mae: 0.0079
Epoch 6/20
78/78 [==============================] - 21s 265ms/step - loss: 8.7917e-05 - mape: 1.7393 - mae: 0.0075 - val_loss: 2.5182e-05 - val_mape: 0.4692 - val_mae: 0.0044
Epoch 7/20
78/78 [==============================] - 1504s 20s/step - loss: 3.7235e-05 - mape: 1.0969 - mae: 0.0048 - val_loss: 4.2230e-0

In [18]:
start_time = time.time()

execute_EEMD_LSTM('Product_0901')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~10 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 22s 259ms/step - loss: 0.0182 - mape: 374235.6875 - mae: 0.0881 - val_loss: 0.0056 - val_mape: 12.0985 - val_mae: 0.0526
Epoch 2/20
78/78 [==============================] - 20s 253ms/step - loss: 0.0121 - mape: 399051.3750 - mae: 0.0729 - val_loss: 0.0065 - val_mape: 14.0230 - val_mae: 0.0609
Epoch 3/20
78/78 [==============================] - 20s 261ms/step - loss: 0.0120 - mape: 428825.5000 - mae: 0.0718 - val_loss: 0.0049 - val_mape: 9.7629 - val_mae: 0.0444
Epoch 4/20
78/78 [==============================] - 20s 253ms/step - loss: 0.0119 - mape: 428909.5938 - mae: 0.0722 - val_loss: 0.0054 - val_mape: 10.8051 - val_mae: 0.0511
Epoch 5/20
78/78 [==============================] - 20s 256ms/step - loss: 0.0113 - mape: 405125.0312 - mae: 0.0673 - val_loss: 0.0048 - val_mape: 9.6507 - val_mae: 0.0434
Epoch 6/20
78/78 [==============================] - 20s 254ms/step - loss: 0.0117 - mape

Epoch 6/20
78/78 [==============================] - 21s 275ms/step - loss: 2.4995e-04 - mape: 54991.0859 - mae: 0.0122 - val_loss: 1.0444e-04 - val_mape: 1.8861 - val_mae: 0.0087
Epoch 7/20
78/78 [==============================] - 21s 274ms/step - loss: 1.5180e-04 - mape: 27181.9180 - mae: 0.0094 - val_loss: 1.6654e-04 - val_mape: 2.4595 - val_mae: 0.0114
Epoch 8/20
78/78 [==============================] - 22s 282ms/step - loss: 1.2495e-04 - mape: 20279.2520 - mae: 0.0083 - val_loss: 6.5592e-05 - val_mape: 1.4872 - val_mae: 0.0066
Epoch 9/20
78/78 [==============================] - 22s 288ms/step - loss: 1.7478e-04 - mape: 30782.5703 - mae: 0.0108 - val_loss: 3.8569e-05 - val_mape: 1.0419 - val_mae: 0.0047
Epoch 10/20
78/78 [==============================] - 21s 265ms/step - loss: 1.3366e-04 - mape: 35540.1719 - mae: 0.0087 - val_loss: 5.5497e-05 - val_mape: 1.3381 - val_mae: 0.0059
Epoch 11/20
78/78 [==============================] - 20s 257ms/step - loss: 9.6920e-05 - mape: 16824.427

Epoch 1/20
78/78 [==============================] - 25s 288ms/step - loss: 0.0181 - mape: 11.6482 - mae: 0.0755 - val_loss: 3.9332e-04 - val_mape: 9.1827 - val_mae: 0.0176
Epoch 2/20
78/78 [==============================] - 22s 282ms/step - loss: 1.2999e-05 - mape: 0.4414 - mae: 0.0027 - val_loss: 8.9670e-04 - val_mape: 14.4111 - val_mae: 0.0285
Epoch 3/20
78/78 [==============================] - 22s 284ms/step - loss: 4.8059e-06 - mape: 0.2935 - mae: 0.0018 - val_loss: 7.4417e-04 - val_mape: 13.0578 - val_mae: 0.0257
Epoch 4/20
11/11 [==============================] - 2s 136ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_10 --------
Epoch 1/20
78/78 [==============================] - 24s 285ms/step - loss: 0.0159 - mape: 156253.0469 - mae: 0.0976 - val_loss: 0.0105 - val_mape: 13.6881 - val_mae: 0.0861
Epoch 2/20
78/78 [==============================] - 22s 282ms/step - loss: 0.0104 - mape: 188302.9062 - mae: 0.0816 - val_loss: 0.0123 - val_mape: 14.3290 - val_mae: 0.0923
Epoch 3/20
78/

In [19]:
start_time = time.time()

execute_EEMD_LSTM('Product_1154')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~11 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 26s 295ms/step - loss: 0.0140 - mape: 19.6146 - mae: 0.0820 - val_loss: 0.0054 - val_mape: 12.5207 - val_mae: 0.0525
Epoch 2/20
78/78 [==============================] - 22s 287ms/step - loss: 0.0079 - mape: 15.8288 - mae: 0.0642 - val_loss: 0.0053 - val_mape: 12.1852 - val_mae: 0.0515
Epoch 3/20
78/78 [==============================] - 23s 289ms/step - loss: 0.0081 - mape: 15.7572 - mae: 0.0639 - val_loss: 0.0057 - val_mape: 12.4836 - val_mae: 0.0558
Epoch 4/20
78/78 [==============================] - 23s 291ms/step - loss: 0.0079 - mape: 15.7182 - mae: 0.0637 - val_loss: 0.0052 - val_mape: 12.0941 - val_mae: 0.0512
Epoch 5/20
78/78 [==============================] - 23s 292ms/step - loss: 0.0079 - mape: 15.6025 - mae: 0.0630 - val_loss: 0.0052 - val_mape: 12.1481 - val_mae: 0.0513
Epoch 6/20
78/78 [==============================] - 22s 287ms/step - loss: 0.0082 - mape: 16.0780 - mae: 0

78/78 [==============================] - 22s 278ms/step - loss: 9.8535e-05 - mape: 1.8549 - mae: 0.0078 - val_loss: 6.3170e-05 - val_mape: 1.5534 - val_mae: 0.0066
Epoch 20/20
11/11 [==============================] - 2s 142ms/step
--------Total: 0~11 eIMFs, Now: eIMFs_3 --------
Epoch 1/20
78/78 [==============================] - 24s 281ms/step - loss: 0.0223 - mape: 38.8965 - mae: 0.1088 - val_loss: 0.0042 - val_mape: 17.0953 - val_mae: 0.0494
Epoch 2/20
78/78 [==============================] - 21s 275ms/step - loss: 0.0023 - mape: 13.5672 - mae: 0.0378 - val_loss: 9.4107e-04 - val_mape: 7.7797 - val_mae: 0.0259
Epoch 3/20
78/78 [==============================] - 22s 278ms/step - loss: 7.8488e-04 - mape: 7.6868 - mae: 0.0223 - val_loss: 6.6820e-04 - val_mape: 6.3736 - val_mae: 0.0215
Epoch 4/20
78/78 [==============================] - 21s 275ms/step - loss: 4.7338e-04 - mape: 5.9688 - mae: 0.0175 - val_loss: 3.7328e-04 - val_mape: 4.6511 - val_mae: 0.0160
Epoch 5/20
78/78 [===========

Epoch 8/20
78/78 [==============================] - 21s 273ms/step - loss: 2.7927e-05 - mape: 1.2506 - mae: 0.0042 - val_loss: 1.5030e-05 - val_mape: 0.8543 - val_mae: 0.0030
Epoch 9/20
78/78 [==============================] - 21s 273ms/step - loss: 2.5812e-05 - mape: 1.2109 - mae: 0.0041 - val_loss: 1.1040e-05 - val_mape: 0.8262 - val_mae: 0.0029
Epoch 10/20
78/78 [==============================] - 21s 271ms/step - loss: 1.4907e-05 - mape: 0.9517 - mae: 0.0031 - val_loss: 4.8212e-05 - val_mape: 1.6965 - val_mae: 0.0063
Epoch 11/20
78/78 [==============================] - 21s 272ms/step - loss: 4.6638e-05 - mape: 1.4984 - mae: 0.0053 - val_loss: 5.3981e-05 - val_mape: 1.9735 - val_mae: 0.0071
Epoch 12/20
11/11 [==============================] - 2s 139ms/step
--------Total: 0~11 eIMFs, Now: eIMFs_6 --------
Epoch 1/20
78/78 [==============================] - 24s 276ms/step - loss: 0.0092 - mape: 22856.0352 - mae: 0.0477 - val_loss: 9.7743e-05 - val_mape: 1.4595 - val_mae: 0.0075
Epoch 2

11/11 [==============================] - 2s 125ms/step
--------Total: 0~11 eIMFs, Now: eIMFs_10 --------
Epoch 1/20
78/78 [==============================] - 25s 293ms/step - loss: 0.0178 - mape: 10.5829 - mae: 0.0676 - val_loss: 9.2693e-04 - val_mape: 13.9050 - val_mae: 0.0287
Epoch 2/20
78/78 [==============================] - 22s 282ms/step - loss: 1.7707e-05 - mape: 0.5434 - mae: 0.0033 - val_loss: 0.0010 - val_mape: 14.6191 - val_mae: 0.0303
Epoch 3/20
78/78 [==============================] - 22s 285ms/step - loss: 7.5027e-06 - mape: 0.3780 - mae: 0.0022 - val_loss: 8.9616e-04 - val_mape: 13.6393 - val_mae: 0.0281
Epoch 4/20
78/78 [==============================] - 22s 288ms/step - loss: 5.3698e-06 - mape: 0.3225 - mae: 0.0019 - val_loss: 8.6127e-04 - val_mape: 13.3590 - val_mae: 0.0276
Epoch 5/20
78/78 [==============================] - 22s 282ms/step - loss: 3.6328e-06 - mape: 0.2619 - mae: 0.0016 - val_loss: 7.2742e-04 - val_mape: 12.1776 - val_mae: 0.0250
Epoch 6/20
78/78 [====

In [20]:
start_time = time.time()

execute_EEMD_LSTM('Product_1248')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~10 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 26s 299ms/step - loss: 0.0152 - mape: 393441.3750 - mae: 0.0796 - val_loss: 0.0030 - val_mape: 9.7288 - val_mae: 0.0446
Epoch 2/20
78/78 [==============================] - 23s 295ms/step - loss: 0.0069 - mape: 387668.5312 - mae: 0.0548 - val_loss: 0.0026 - val_mape: 7.5618 - val_mae: 0.0370
Epoch 3/20
78/78 [==============================] - 23s 293ms/step - loss: 0.0067 - mape: 380041.7500 - mae: 0.0536 - val_loss: 0.0024 - val_mape: 7.5404 - val_mae: 0.0362
Epoch 4/20
78/78 [==============================] - 23s 295ms/step - loss: 0.0068 - mape: 377687.1250 - mae: 0.0549 - val_loss: 0.0027 - val_mape: 8.7439 - val_mae: 0.0405
Epoch 5/20
78/78 [==============================] - 23s 295ms/step - loss: 0.0067 - mape: 382880.5625 - mae: 0.0544 - val_loss: 0.0026 - val_mape: 7.5302 - val_mae: 0.0368
Epoch 6/20
11/11 [==============================] - 2s 132ms/step
--------Total: 0~10 eIMFs

Epoch 8/20
78/78 [==============================] - 31s 394ms/step - loss: 2.0629e-04 - mape: 11008.3359 - mae: 0.0108 - val_loss: 1.9959e-05 - val_mape: 0.7264 - val_mae: 0.0035
Epoch 9/20
78/78 [==============================] - 31s 395ms/step - loss: 2.8534e-04 - mape: 14951.9336 - mae: 0.0131 - val_loss: 1.0315e-04 - val_mape: 1.8330 - val_mae: 0.0090
Epoch 10/20
78/78 [==============================] - 31s 394ms/step - loss: 2.1850e-04 - mape: 8705.8271 - mae: 0.0113 - val_loss: 1.9593e-04 - val_mape: 2.5628 - val_mae: 0.0128
Epoch 11/20
11/11 [==============================] - 2s 170ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_4 --------
Epoch 1/20
78/78 [==============================] - 33s 398ms/step - loss: 0.0145 - mape: 77371.6719 - mae: 0.0761 - val_loss: 0.0012 - val_mape: 7.3176 - val_mae: 0.0340
Epoch 2/20
78/78 [==============================] - 31s 392ms/step - loss: 6.8573e-04 - mape: 90231.3203 - mae: 0.0199 - val_loss: 2.0427e-04 - val_mape: 2.9986 - val_mae: 0.01

78/78 [==============================] - 29s 371ms/step - loss: 0.0284 - mape: 376028.7812 - mae: 0.1351 - val_loss: 0.0275 - val_mape: 32.4266 - val_mae: 0.1276
Epoch 3/20
78/78 [==============================] - 31s 393ms/step - loss: 0.0277 - mape: 330343.1250 - mae: 0.1334 - val_loss: 0.0321 - val_mape: 32.0101 - val_mae: 0.1397
Epoch 4/20
78/78 [==============================] - 29s 372ms/step - loss: 0.0284 - mape: 395360.9688 - mae: 0.1362 - val_loss: 0.0267 - val_mape: 33.7260 - val_mae: 0.1261
Epoch 5/20
78/78 [==============================] - 29s 373ms/step - loss: 0.0278 - mape: 375593.3438 - mae: 0.1339 - val_loss: 0.0274 - val_mape: 32.5580 - val_mae: 0.1272
Epoch 6/20
78/78 [==============================] - 29s 369ms/step - loss: 0.0281 - mape: 337186.5000 - mae: 0.1343 - val_loss: 0.0383 - val_mape: 33.2210 - val_mae: 0.1558
Epoch 7/20
11/11 [==============================] - 2s 160ms/step
실행 시간: 43.45 분


In [21]:
start_time = time.time()

execute_EEMD_LSTM('Product_1295')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~10 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 31s 363ms/step - loss: 0.0228 - mape: 369129.0625 - mae: 0.1133 - val_loss: 0.0138 - val_mape: 22.4040 - val_mae: 0.0972
Epoch 2/20
78/78 [==============================] - 28s 354ms/step - loss: 0.0141 - mape: 362685.7812 - mae: 0.0931 - val_loss: 0.0138 - val_mape: 22.8904 - val_mae: 0.0977
Epoch 3/20
78/78 [==============================] - 28s 353ms/step - loss: 0.0140 - mape: 372794.6875 - mae: 0.0944 - val_loss: 0.0168 - val_mape: 21.1048 - val_mae: 0.1033
Epoch 4/20
11/11 [==============================] - 2s 150ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_1 --------
Epoch 1/20
78/78 [==============================] - 30s 357ms/step - loss: 0.0221 - mape: 356340.6875 - mae: 0.1119 - val_loss: 0.0136 - val_mape: 24.6412 - val_mae: 0.0967
Epoch 2/20
78/78 [==============================] - 28s 357ms/step - loss: 0.0132 - mape: 412140.8125 - mae: 0.0931 - val_loss: 0.0135 - val_map

Epoch 7/20
78/78 [==============================] - 28s 356ms/step - loss: 2.2197e-04 - mape: 3.4544 - mae: 0.0120 - val_loss: 4.0542e-04 - val_mape: 46949.7969 - val_mae: 0.0130
Epoch 8/20
78/78 [==============================] - 28s 357ms/step - loss: 1.0221e-04 - mape: 2.3434 - mae: 0.0078 - val_loss: 4.1455e-04 - val_mape: 145634.7656 - val_mae: 0.0146
Epoch 9/20
78/78 [==============================] - 28s 360ms/step - loss: 1.1355e-04 - mape: 2.5964 - mae: 0.0083 - val_loss: 3.8075e-04 - val_mape: 179798.1094 - val_mae: 0.0122
Epoch 10/20
78/78 [==============================] - 28s 356ms/step - loss: 9.0396e-05 - mape: 1.8703 - mae: 0.0075 - val_loss: 3.2613e-04 - val_mape: 89711.3203 - val_mae: 0.0113
Epoch 11/20
78/78 [==============================] - 28s 358ms/step - loss: 9.6016e-05 - mape: 2.1614 - mae: 0.0077 - val_loss: 3.7438e-04 - val_mape: 193359.8594 - val_mae: 0.0113
Epoch 12/20
78/78 [==============================] - 28s 357ms/step - loss: 1.1370e-04 - mape: 2.314

Epoch 2/20
78/78 [==============================] - 31s 394ms/step - loss: 5.6021e-05 - mape: 4102.8071 - mae: 0.0058 - val_loss: 1.2974e-04 - val_mape: 3.0871 - val_mae: 0.0113
Epoch 3/20
78/78 [==============================] - 31s 394ms/step - loss: 5.5658e-05 - mape: 758.4232 - mae: 0.0058 - val_loss: 2.0723e-04 - val_mape: 3.8674 - val_mae: 0.0143
Epoch 4/20
78/78 [==============================] - 31s 395ms/step - loss: 6.7836e-05 - mape: 2853.0173 - mae: 0.0064 - val_loss: 1.3431e-04 - val_mape: 3.1658 - val_mae: 0.0115
Epoch 5/20
11/11 [==============================] - 2s 163ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_8 --------
Epoch 1/20
78/78 [==============================] - 33s 398ms/step - loss: 0.0123 - mape: 39.9089 - mae: 0.0417 - val_loss: 1.2989e-05 - val_mape: 0.3712 - val_mae: 0.0036
Epoch 2/20
78/78 [==============================] - 30s 391ms/step - loss: 4.9355e-06 - mape: 3.0606 - mae: 0.0017 - val_loss: 6.0609e-07 - val_mape: 0.0801 - val_mae: 7.7790e-04
E

In [22]:
start_time = time.time()

execute_EEMD_LSTM('Product_1378')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~10 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 35s 413ms/step - loss: 0.0135 - mape: 382565.2500 - mae: 0.0826 - val_loss: 0.0142 - val_mape: 26.1150 - val_mae: 0.0921
Epoch 2/20
78/78 [==============================] - 32s 408ms/step - loss: 0.0090 - mape: 336570.8438 - mae: 0.0728 - val_loss: 0.0144 - val_mape: 26.9288 - val_mae: 0.0942
Epoch 3/20
78/78 [==============================] - 32s 407ms/step - loss: 0.0086 - mape: 378233.8438 - mae: 0.0715 - val_loss: 0.0141 - val_mape: 26.1644 - val_mae: 0.0922
Epoch 4/20
78/78 [==============================] - 32s 408ms/step - loss: 0.0086 - mape: 367137.5312 - mae: 0.0713 - val_loss: 0.0143 - val_mape: 23.0906 - val_mae: 0.0881
Epoch 5/20
78/78 [==============================] - 32s 406ms/step - loss: 0.0085 - mape: 395818.5938 - mae: 0.0707 - val_loss: 0.0140 - val_mape: 25.8281 - val_mae: 0.0913
Epoch 6/20
78/78 [==============================] - 32s 408ms/step - loss: 0.0087 - ma

Epoch 10/20
78/78 [==============================] - 30s 387ms/step - loss: 1.2772e-04 - mape: 8968.6201 - mae: 0.0089 - val_loss: 5.5668e-05 - val_mape: 1.5210 - val_mae: 0.0059
Epoch 11/20
78/78 [==============================] - 30s 386ms/step - loss: 1.1276e-04 - mape: 17410.5684 - mae: 0.0082 - val_loss: 7.9247e-05 - val_mape: 1.9416 - val_mae: 0.0074
Epoch 12/20
78/78 [==============================] - 30s 384ms/step - loss: 9.8220e-05 - mape: 8967.7646 - mae: 0.0078 - val_loss: 5.3826e-05 - val_mape: 1.4386 - val_mae: 0.0057
Epoch 13/20
78/78 [==============================] - 30s 386ms/step - loss: 1.4319e-04 - mape: 16082.2090 - mae: 0.0096 - val_loss: 1.1872e-04 - val_mape: 2.3458 - val_mae: 0.0090
Epoch 14/20
78/78 [==============================] - 30s 384ms/step - loss: 9.9011e-05 - mape: 30399.5176 - mae: 0.0077 - val_loss: 5.2300e-05 - val_mape: 1.4721 - val_mae: 0.0054
Epoch 15/20
78/78 [==============================] - 30s 388ms/step - loss: 7.4092e-05 - mape: 3174.57

78/78 [==============================] - 30s 352ms/step - loss: 0.0074 - mape: 50.8850 - mae: 0.0385 - val_loss: 1.7105e-04 - val_mape: 1.4756 - val_mae: 0.0131
Epoch 2/20
78/78 [==============================] - 27s 345ms/step - loss: 9.5187e-06 - mape: 2.6868 - mae: 0.0022 - val_loss: 4.3848e-07 - val_mape: 0.0700 - val_mae: 6.1234e-04
Epoch 3/20
78/78 [==============================] - 27s 345ms/step - loss: 2.3315e-07 - mape: 0.4943 - mae: 3.7993e-04 - val_loss: 1.8028e-06 - val_mape: 0.1463 - val_mae: 0.0013
Epoch 4/20
78/78 [==============================] - 27s 345ms/step - loss: 1.4005e-07 - mape: 0.4554 - mae: 2.7747e-04 - val_loss: 1.4818e-06 - val_mape: 0.1313 - val_mae: 0.0012
Epoch 5/20
11/11 [==============================] - 2s 152ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_10 --------
Epoch 1/20
78/78 [==============================] - 29s 338ms/step - loss: 0.0244 - mape: 27.6089 - mae: 0.1184 - val_loss: 0.0224 - val_mape: 56.2223 - val_mae: 0.1246
Epoch 2/20
78/78 

In [23]:
start_time = time.time()

execute_EEMD_LSTM('Product_2004')

elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))

--------Total: 0~10 eIMFs, Now: eIMFs_0 --------
Epoch 1/20
78/78 [==============================] - 33s 397ms/step - loss: 0.0197 - mape: 343714.4688 - mae: 0.1002 - val_loss: 0.0127 - val_mape: 20.8171 - val_mae: 0.0828
Epoch 2/20
78/78 [==============================] - 30s 386ms/step - loss: 0.0140 - mape: 351806.7188 - mae: 0.0851 - val_loss: 0.0136 - val_mape: 20.1527 - val_mae: 0.0887
Epoch 3/20
78/78 [==============================] - 30s 384ms/step - loss: 0.0139 - mape: 351641.9062 - mae: 0.0843 - val_loss: 0.0138 - val_mape: 23.1605 - val_mae: 0.0891
Epoch 4/20
11/11 [==============================] - 2s 160ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_1 --------
Epoch 1/20
78/78 [==============================] - 34s 410ms/step - loss: 0.0154 - mape: 419096.8750 - mae: 0.0844 - val_loss: 0.0052 - val_mape: 12.1322 - val_mae: 0.0589
Epoch 2/20
78/78 [==============================] - 31s 403ms/step - loss: 0.0091 - mape: 483253.0312 - mae: 0.0702 - val_loss: 0.0062 - val_map

78/78 [==============================] - 34s 439ms/step - loss: 1.4838e-04 - mape: 14475.9180 - mae: 0.0096 - val_loss: 1.0044e-04 - val_mape: 2.0388 - val_mae: 0.0082
Epoch 12/20
78/78 [==============================] - 34s 438ms/step - loss: 1.0866e-04 - mape: 8256.6436 - mae: 0.0082 - val_loss: 1.9909e-04 - val_mape: 2.8062 - val_mae: 0.0121
Epoch 13/20
78/78 [==============================] - 34s 437ms/step - loss: 1.0525e-04 - mape: 7423.2529 - mae: 0.0081 - val_loss: 9.7366e-05 - val_mape: 1.9889 - val_mae: 0.0083
Epoch 14/20
78/78 [==============================] - 34s 440ms/step - loss: 1.6093e-04 - mape: 7645.1821 - mae: 0.0094 - val_loss: 4.1099e-04 - val_mape: 5.0005 - val_mae: 0.0193
Epoch 15/20
78/78 [==============================] - 34s 438ms/step - loss: 1.5338e-04 - mape: 21242.6328 - mae: 0.0097 - val_loss: 7.3708e-05 - val_mape: 1.8362 - val_mae: 0.0072
Epoch 16/20
78/78 [==============================] - 34s 438ms/step - loss: 1.0188e-04 - mape: 12103.6533 - mae: 0.

78/78 [==============================] - 36s 456ms/step - loss: 3.0526e-05 - mape: 3144.1584 - mae: 0.0040 - val_loss: 9.0208e-05 - val_mape: 1.2696 - val_mae: 0.0094
Epoch 6/20
78/78 [==============================] - 36s 455ms/step - loss: 3.0893e-05 - mape: 5573.7671 - mae: 0.0042 - val_loss: 1.7557e-04 - val_mape: 1.7629 - val_mae: 0.0130
Epoch 7/20
78/78 [==============================] - 36s 458ms/step - loss: 7.4348e-05 - mape: 4524.1396 - mae: 0.0066 - val_loss: 2.1855e-04 - val_mape: 1.9792 - val_mae: 0.0148
Epoch 8/20
11/11 [==============================] - 2s 186ms/step
--------Total: 0~10 eIMFs, Now: eIMFs_9 --------
Epoch 1/20
78/78 [==============================] - 38s 462ms/step - loss: 0.0083 - mape: 24.0719 - mae: 0.0383 - val_loss: 4.8525e-07 - val_mape: 0.0637 - val_mae: 5.6615e-04
Epoch 2/20
78/78 [==============================] - 35s 446ms/step - loss: 1.0608e-06 - mape: 0.7222 - mae: 6.6694e-04 - val_loss: 6.3639e-06 - val_mape: 0.2790 - val_mae: 0.0025
Epoch 3